In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

# Generate synthetic data for five assets
assets = [
    {"name": "Asset1", "expected_return": 0.10, "risk": 0.15},
    {"name": "Asset2", "expected_return": 0.12, "risk": 0.20},
    {"name": "Asset3", "expected_return": 0.08, "risk": 0.10},
    {"name": "Asset4", "expected_return": 0.09, "risk": 0.12},
    {"name": "Asset5", "expected_return": 0.11, "risk": 0.18},
]


def portfolio_performance(weights, assets):
    portfolio_return = sum(asset["expected_return"] * weight for asset, weight in zip(assets, weights))
    portfolio_risk = np.sqrt(sum((asset["risk"] * weight) ** 2 for asset, weight in zip(assets, weights)))
    return portfolio_return, portfolio_risk


# Set up genetic algorithm
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  # Maximize return, minimize risk
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=len(assets))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evaluate_portfolio(individual):
    weights = [max(weight, 0) for weight in individual]  # Ensure non-negative weights
    weights = [weight / sum(weights) for weight in weights]  # Normalize weights
    portfolio_return, portfolio_risk = portfolio_performance(weights, assets)
    return portfolio_return, portfolio_risk


toolbox.register("evaluate", evaluate_portfolio)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutPolynomialBounded, low=0, up=1, eta=1.0, indpb=0.2)
toolbox.register("select", tools.selNSGA2)


def repair_weights(individual):
    weights = [max(weight, 0) for weight in individual]  # Ensure non-negative weights
    total = sum(weights)
    if total > 0:
        for i in range(len(individual)):
            individual[i] = weights[i] / total  # Normalize weights


# Run genetic algorithm
population = toolbox.population(n=50)
n_generations = 40

for gen in range(n_generations):
    print(f"\nGeneration {gen + 1}")

    # Evaluate the individuals
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.7:
            toolbox.mate(child1, child2)
            repair_weights(child1)
            repair_weights(child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            repair_weights(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    population[:] = offspring

    # Print the statistics for the current generation
    for ind in population:
        weights = [weight / sum(ind) for weight in ind]
        print(f"Portfolio weights: {weights}, Return: {ind.fitness.values[0]}, Risk: {ind.fitness.values[1]}")

# Get the final population
final_population = tools.selBest(population, k=len(population))

# Get the best individual
best_ind = final_population[0]
best_weights = [weight / sum(best_ind) for weight in best_ind]
best_return, best_risk = portfolio_performance(best_weights, assets)
print(f"\nBest Portfolio Weights: {best_weights}")
print(f"Best Portfolio Return: {best_return}, Risk: {best_risk}")

# Visualize the best portfolio
asset_names = [asset["name"] for asset in assets]
plt.pie(best_weights, labels=asset_names, autopct='%1.1f%%', startangle=140)
plt.title('Optimal Portfolio Allocation')
plt.show()